In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 128
test_batch = 100
lr = 0.1
schedule = [20, 125, 225, 275]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b1' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 300] LR: 0.100000
1/538 Data:1.835 | Batch:6.062 | Total:0:00:06 | ETA:0:54:17 | Loss:0.7030371427536011 | top1:49.21875
11/538 Data:0.002 | Batch:0.522 | Total:0:00:11 | ETA:0:09:30 | Loss:5.848791057413274 | top1:48.934661865234375
21/538 Data:0.001 | Batch:0.253 | Total:0:00:15 | ETA:0:03:59 | Loss:4.218069888296581 | top1:48.7351188659668
31/538 Data:0.001 | Batch:0.284 | Total:0:00:18 | ETA:0:02:35 | Loss:3.172889023057876 | top1:49.369956970214844
41/538 Data:0.002 | Batch:0.436 | Total:0:00:22 | ETA:0:02:59 | Loss:2.58459990780528 | top1:49.42835235595703
51/538 Data:0.001 | Batch:0.286 | Total:0:00:25 | ETA:0:02:56 | Loss:2.221950199089798 | top1:49.2493896484375
61/538 Data:0.001 | Batch:0.443 | Total:0:00:29 | ETA:0:02:42 | Loss:1.9800909474247792 | top1:49.18032455444336
71/538 Data:0.002 | Batch:0.527 | Total:0:00:33 | ETA:0:03:09 | Loss:1.8029997071749728 | top1:49.361793518066406
81/538 Data:0.002 | Batch:0.541 | Total:0:00:38 | ETA:0:04:00 | Loss:1.675801403

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


78/78 Data:0.002 | Batch:0.114 | Total:0:00:14 | ETA:0:00:00 | Loss:0.6961274903554183 | top1:50.0

Epoch: [2 | 300] LR: 0.170000
1/538 Data:1.260 | Batch:1.766 | Total:0:00:01 | ETA:0:15:50 | Loss:0.7032731771469116 | top1:47.65625
11/538 Data:0.002 | Batch:0.545 | Total:0:00:06 | ETA:0:05:39 | Loss:0.7109143408862028 | top1:50.14204788208008
21/538 Data:0.014 | Batch:0.577 | Total:0:00:12 | ETA:0:04:35 | Loss:0.7107704508872259 | top1:49.5163688659668
31/538 Data:0.001 | Batch:0.535 | Total:0:00:17 | ETA:0:04:22 | Loss:0.7078034300957957 | top1:50.126007080078125
41/538 Data:0.001 | Batch:0.506 | Total:0:00:22 | ETA:0:04:18 | Loss:0.7058350778207546 | top1:50.89557647705078
51/538 Data:0.002 | Batch:0.524 | Total:0:00:27 | ETA:0:04:13 | Loss:0.7076532665420981 | top1:50.64338302612305
61/538 Data:0.002 | Batch:0.526 | Total:0:00:32 | ETA:0:04:08 | Loss:0.7056429200485105 | top1:50.76844024658203
71/538 Data:0.002 | Batch:0.517 | Total:0:00:38 | ETA:0:04:07 | Loss:0.7047330384523096 |

161/538 Data:0.001 | Batch:0.501 | Total:0:01:25 | ETA:0:03:18 | Loss:0.6962834430037078 | top1:50.3105583190918
171/538 Data:0.002 | Batch:0.504 | Total:0:01:30 | ETA:0:03:11 | Loss:0.6963113302375838 | top1:50.13249206542969
181/538 Data:0.001 | Batch:0.520 | Total:0:01:35 | ETA:0:03:07 | Loss:0.6962871742511981 | top1:50.14675521850586
191/538 Data:0.007 | Batch:0.538 | Total:0:01:41 | ETA:0:02:58 | Loss:0.6961281702156467 | top1:50.22087860107422
201/538 Data:0.001 | Batch:0.521 | Total:0:01:46 | ETA:0:02:59 | Loss:0.6963640798383685 | top1:50.2681884765625
211/538 Data:0.002 | Batch:0.528 | Total:0:01:51 | ETA:0:02:49 | Loss:0.696404489013256 | top1:50.370262145996094
221/538 Data:0.001 | Batch:0.505 | Total:0:01:56 | ETA:0:02:46 | Loss:0.6963135910249943 | top1:50.36764907836914
231/538 Data:0.002 | Batch:0.493 | Total:0:02:01 | ETA:0:02:40 | Loss:0.6962309118473169 | top1:50.436283111572266
241/538 Data:0.001 | Batch:0.534 | Total:0:02:06 | ETA:0:02:33 | Loss:0.6961655463420504 

331/538 Data:0.001 | Batch:0.522 | Total:0:02:53 | ETA:0:01:48 | Loss:0.6955491686155428 | top1:50.13925552368164
341/538 Data:0.013 | Batch:0.580 | Total:0:02:59 | ETA:0:01:46 | Loss:0.6955100449998358 | top1:50.14433670043945
351/538 Data:0.002 | Batch:0.491 | Total:0:03:04 | ETA:0:01:38 | Loss:0.6954677779110748 | top1:50.14690017700195
361/538 Data:0.012 | Batch:0.553 | Total:0:03:09 | ETA:0:01:33 | Loss:0.6954421845169279 | top1:50.10171127319336
371/538 Data:0.001 | Batch:0.503 | Total:0:03:15 | ETA:0:01:28 | Loss:0.6954728853349095 | top1:50.09897232055664
381/538 Data:0.001 | Batch:0.529 | Total:0:03:20 | ETA:0:01:22 | Loss:0.6954577868691892 | top1:50.094322204589844
391/538 Data:0.000 | Batch:0.509 | Total:0:03:25 | ETA:0:01:17 | Loss:0.6954105324147607 | top1:50.11788558959961
401/538 Data:0.001 | Batch:0.499 | Total:0:03:30 | ETA:0:01:12 | Loss:0.6954245261123353 | top1:50.062347412109375
411/538 Data:0.007 | Batch:0.513 | Total:0:03:35 | ETA:0:01:06 | Loss:0.69538846807758

...

311/538 Data:0.001 | Batch:0.552 | Total:0:02:45 | ETA:0:02:00 | Loss:0.6854192882872088 | top1:55.54410934448242
321/538 Data:0.001 | Batch:0.527 | Total:0:02:51 | ETA:0:01:54 | Loss:0.6850493407323724 | top1:55.62937927246094
331/538 Data:0.001 | Batch:0.563 | Total:0:02:56 | ETA:0:01:49 | Loss:0.6848512902360668 | top1:55.683536529541016
341/538 Data:0.002 | Batch:0.600 | Total:0:03:01 | ETA:0:01:44 | Loss:0.684428127979603 | top1:55.75971221923828
351/538 Data:0.001 | Batch:0.504 | Total:0:03:06 | ETA:0:01:43 | Loss:0.6843340953870377 | top1:55.7937126159668
361/538 Data:0.002 | Batch:0.504 | Total:0:03:12 | ETA:0:01:33 | Loss:0.6840803200518325 | top1:55.83881378173828
371/538 Data:0.001 | Batch:0.541 | Total:0:03:17 | ETA:0:01:30 | Loss:0.6834895125617878 | top1:55.96150588989258
381/538 Data:0.002 | Batch:0.525 | Total:0:03:22 | ETA:0:01:22 | Loss:0.6832200432699809 | top1:56.028541564941406
391/538 Data:0.006 | Batch:0.499 | Total:0:03:28 | ETA:0:01:20 | Loss:0.683071259342496 

481/538 Data:0.001 | Batch:0.516 | Total:0:04:11 | ETA:0:00:30 | Loss:0.6363909123097536 | top1:64.19243621826172
491/538 Data:0.001 | Batch:0.530 | Total:0:04:16 | ETA:0:00:25 | Loss:0.6360424795849988 | top1:64.22638702392578
501/538 Data:0.002 | Batch:0.515 | Total:0:04:21 | ETA:0:00:20 | Loss:0.6351495119863879 | top1:64.3198013305664
511/538 Data:0.012 | Batch:0.558 | Total:0:04:27 | ETA:0:00:15 | Loss:0.6348706690299301 | top1:64.3621597290039
521/538 Data:0.001 | Batch:0.512 | Total:0:04:32 | ETA:0:00:10 | Loss:0.6338356139563782 | top1:64.4478759765625
531/538 Data:0.001 | Batch:0.542 | Total:0:04:37 | ETA:0:00:04 | Loss:0.6328422781886803 | top1:64.53919219970703
78/78 Data:0.002 | Batch:0.134 | Total:0:00:16 | ETA:0:00:00 | Loss:0.4947199026743571 | top1:77.4102554321289

Epoch: [10 | 300] LR: 0.730000
1/538 Data:1.453 | Batch:1.986 | Total:0:00:01 | ETA:0:17:47 | Loss:0.6191967129707336 | top1:63.28125
11/538 Data:0.002 | Batch:0.557 | Total:0:00:07 | ETA:0:05:53 | Loss:0.61

101/538 Data:0.002 | Batch:0.508 | Total:0:00:54 | ETA:0:03:54 | Loss:0.5633611720387298 | top1:71.43409729003906
111/538 Data:0.002 | Batch:0.553 | Total:0:00:59 | ETA:0:03:43 | Loss:0.562519777465511 | top1:71.44566345214844
121/538 Data:0.003 | Batch:0.522 | Total:0:01:05 | ETA:0:03:51 | Loss:0.5629971465788597 | top1:71.27452850341797
131/538 Data:0.002 | Batch:0.512 | Total:0:01:10 | ETA:0:03:31 | Loss:0.5645442377519971 | top1:71.06393432617188
141/538 Data:0.003 | Batch:0.526 | Total:0:01:15 | ETA:0:03:31 | Loss:0.5637233498248648 | top1:71.18794250488281
151/538 Data:0.001 | Batch:0.518 | Total:0:01:21 | ETA:0:03:24 | Loss:0.563610223350146 | top1:71.13513946533203
161/538 Data:0.001 | Batch:0.504 | Total:0:01:26 | ETA:0:03:22 | Loss:0.5633579960891179 | top1:71.1956558227539
171/538 Data:0.001 | Batch:0.515 | Total:0:01:31 | ETA:0:03:14 | Loss:0.5635844341844146 | top1:71.13943481445312
181/538 Data:0.001 | Batch:0.517 | Total:0:01:36 | ETA:0:03:09 | Loss:0.5612652095971187 | 

281/538 Data:0.012 | Batch:0.521 | Total:0:02:32 | ETA:0:02:15 | Loss:0.5321543198675448 | top1:73.59319305419922
291/538 Data:0.002 | Batch:0.507 | Total:0:02:37 | ETA:0:02:10 | Loss:0.5320271426664595 | top1:73.59590148925781
301/538 Data:0.002 | Batch:0.522 | Total:0:02:42 | ETA:0:01:47 | Loss:0.5319807028453611 | top1:73.59063720703125
311/538 Data:0.001 | Batch:0.505 | Total:0:02:47 | ETA:0:02:06 | Loss:0.532275075889477 | top1:73.59073638916016
321/538 Data:0.010 | Batch:0.512 | Total:0:02:53 | ETA:0:01:55 | Loss:0.5318141618621683 | top1:73.56892395019531
331/538 Data:0.001 | Batch:0.526 | Total:0:02:58 | ETA:0:01:44 | Loss:0.5311747512068273 | top1:73.6404800415039
341/538 Data:0.007 | Batch:0.612 | Total:0:03:03 | ETA:0:01:43 | Loss:0.5307123235418649 | top1:73.70784759521484
351/538 Data:0.002 | Batch:0.529 | Total:0:03:08 | ETA:0:01:39 | Loss:0.5308229451165919 | top1:73.69123840332031
361/538 Data:0.001 | Batch:0.524 | Total:0:03:13 | ETA:0:01:36 | Loss:0.5301946088712962 |

461/538 Data:0.003 | Batch:0.540 | Total:0:04:07 | ETA:0:00:43 | Loss:0.5155890657689721 | top1:74.65767669677734
471/538 Data:0.001 | Batch:0.527 | Total:0:04:12 | ETA:0:00:35 | Loss:0.5147879543198142 | top1:74.7296371459961
481/538 Data:0.013 | Batch:0.533 | Total:0:04:18 | ETA:0:00:32 | Loss:0.5148786011455956 | top1:74.73037719726562
491/538 Data:0.002 | Batch:0.510 | Total:0:04:23 | ETA:0:00:25 | Loss:0.5144136737417544 | top1:74.78042602539062
